<a href="https://colab.research.google.com/github/gopichandchalla16/infosys-internship-real-time-industry-insight-system/blob/janmejay-dev/Realtime_Industry_Insight_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
%%javascript
Jupyter.notebook.metadata.widgets = {}


<IPython.core.display.Javascript object>

# Infosys Springboard Internship Project
## Real-Time Industry Insight & Strategic Intelligence System
### Sprint 1 – Data Sourcing & Handling
**Team Members:** Gopichand, Anshika, Janmejay, Vaishnavi

**Objective:** Collect, clean, and visualize real-time company or market data from external open sources.


STEP:0

In [109]:
# Core installs (quiet to reduce logs)
!pip -q uninstall -y prophet cmdstanpy
!pip -q install wikipedia beautifulsoup4 lxml requests feedparser yfinance
!pip -q install transformers sentencepiece torch
!pip -q install prophet ipywidgets 'numpy<2' cmdstanpy

# Verify cmdstanpy installation
try:
    import cmdstanpy
    print(f"✅ cmdstanpy version: {cmdstanpy.__version__} installed.")
except ImportError:
    print("❌ cmdstanpy not found after installation attempt.")

# Enable widgets in Colab (may require reloading UI once)
from google.colab import output
output.enable_custom_widget_manager()

✅ cmdstanpy version: 1.3.0 installed.


Imports & Global Theme

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import wikipedia
from bs4 import BeautifulSoup
import requests
import feedparser
import urllib.parse
import yfinance as yf
from IPython.display import display, HTML, Image, clear_output

from statsmodels.tsa.arima.model import ARIMA

# HuggingFace sentiment
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

import ipywidgets as widgets

sns.set_theme(style="whitegrid")
plt.rcParams["figure.dpi"] = 120


2) Input: Company Name

In [111]:
company_name = input("Enter company name (e.g., Infosys, TCS, Netflix): ").strip().title()
if not company_name:
    company_name = "Infosys"
print("You entered:", company_name)


Enter company name (e.g., Infosys, TCS, Netflix): meta
You entered: Meta


3) Wikipedia: Summary + Infobox (CEO, Founded, HQ, Revenue, Employees, Logo)

In [112]:
def get_company_info(company_name: str):
    """
    Returns a dict with Wikipedia summary + key infobox fields.
    Uses exact->fuzzy->fallback to avoid 'Infos' issue for 'Infosys', etc.
    """
    try:
        q = company_name.strip().title()
        results = wikipedia.search(q)
        if not results:
            return {"error": f"No Wikipedia pages found for {q}"}

        # 1) Exact match
        page_title = None
        for r in results:
            if r.lower() == q.lower():
                page_title = r
                break

        # 2) Fuzzy best match
        if not page_title:
            match = difflib.get_close_matches(q, results, n=1, cutoff=0.4)
            page_title = match[0] if match else results[0]

        # Fetch page
        page = wikipedia.page(page_title, auto_suggest=False)
        summary = page.summary
        url = page.url

        # Scrape infobox
        html = requests.get(url, timeout=15).text
        soup = BeautifulSoup(html, "lxml")
        infobox = soup.find("table", {"class": "infobox"})

        info = {
            "Company": page.title,
            "Summary": summary,
            "Wikipedia_URL": url,
            "Logo": None,
            "Founded": None,
            "Founders": None,
            "CEO": None,
            "Headquarters": None,
            "Revenue": None,
            "Employees": None
        }

        if infobox:
            img = infobox.find("img")
            if img and img.get("src"):
                src = img["src"]
                info["Logo"] = "https:" + src if src.startswith("//") else src

            for row in infobox.find_all("tr"):
                th = row.find("th")
                td = row.find("td")
                if not th or not td:
                    continue
                key = th.get_text(strip=True).lower()
                val = td.get_text(" ", strip=True)

                if "founded" in key:
                    info["Founded"] = val
                elif "founder" in key:
                    info["Founders"] = val
                elif "headquarters" in key:
                    info["Headquarters"] = val
                elif "revenue" in key:
                    info["Revenue"] = val
                elif "ceo" in key or "chief executive" in key:
                    info["CEO"] = val
                elif "employees" in key:
                    info["Employees"] = val

        return info

    except wikipedia.exceptions.DisambiguationError as e:
        # Choose closest option recursively
        for opt in e.options:
            if company_name.lower() in opt.lower():
                return get_company_info(opt)
        return get_company_info(e.options[0])

    except Exception as ex:
        return {"error": str(ex)}

info = get_company_info(company_name)

# Display the info card
if "error" in info:
    print("Error:", info["error"])
else:
    if info.get("Logo"):
        display(Image(url=info["Logo"], width=160))

    html = f"""
    <div style="font-family: Inter, Arial, sans-serif; max-width: 980px; border-radius: 16px; padding: 20px;
                box-shadow: 0 8px 24px rgba(0,0,0,0.12); border: 1px solid #eee;">
        <h2 style="margin: 0 0 10px 0;">{info.get('Company','')}</h2>
        <p style="color:#fff; line-height:1.6; margin-top:6px;">{info.get('Summary','')}</p>
        <div style="display:grid; grid-template-columns: repeat(auto-fit, minmax(220px, 1fr)); gap:14px; margin-top:14px;">
            <div><b>CEO:</b> {info.get('CEO','—')}</div>
            <div><b>Founded:</b> {info.get('Founded','—')}</div>
            <div><b>Headquarters:</b> {info.get('Headquarters','—')}</div>
            <div><b>Revenue:</b> {info.get('Revenue','—')}</div>
            <div><b>Employees:</b> {info.get('Employees','—')}</div>
        </div>
        <div style="margin-top:12px;">
            <a href="{info.get('Wikipedia_URL','')}" target="_blank">Wikipedia Page →</a>
        </div>
    </div>
    """
    display(HTML(html))


4) Google News: Latest Headlines (RSS)

In [113]:
encoded = urllib.parse.quote(company_name)
news_url = f"https://news.google.com/rss/search?q={encoded}&hl=en-US&gl=US&ceid=US:en"
feed = feedparser.parse(news_url)

print(f"\n📰 Latest Google News for {company_name}:\n")
news_items = []
for entry in feed.entries[:12]:
    title = entry.title
    link = entry.link
    source = getattr(entry, "source", {})
    source_title = source.get("title", "N/A")
    news_items.append({"title": title, "link": link, "source": source_title})
    display(HTML(f'• <a href="{link}" target="_blank">{title}</a> <span style="color:#777">({source_title})</span>'))

# If no items, let the user know
if not news_items:
    print("No recent news found. Try a broader query.")



📰 Latest Google News for Meta:



5) Collect Texts for Sentiment (Headlines as proxy for tweets)

In [114]:
# Use news headlines as text for sentiment analysis
texts = [n["title"] for n in news_items] if news_items else [
    f"{company_name} delivers strong quarterly results",
    f"{company_name} faces regulatory scrutiny over compliance",
    f"Analysts remain neutral on {company_name} outlook"
]
print(f"Using {len(texts)} texts for sentiment analysis.")


Using 12 texts for sentiment analysis.


6) Hugging Face Sentiment (Twitter RoBERTa)

In [115]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["Negative", "Neutral", "Positive"]

def hf_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    with torch.no_grad():
        outputs = hf_model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()[0]
    sentiment = labels[int(np.argmax(probs))]
    return sentiment, probs

sent_rows = []
for t in texts:
    s_label, s_probs = hf_sentiment(t)
    sent_rows.append({
        "text": t,
        "sentiment": s_label,
        "neg": float(s_probs[0]),
        "neu": float(s_probs[1]),
        "pos": float(s_probs[2]),
    })

sent_df = pd.DataFrame(sent_rows)
display(sent_df.head())


,text,sentiment,neg,neu,pos
0,Meta chief AI scientist Yann LeCun plans to ex...,Neutral,0.042439,0.911715,0.045847
1,AI cloud firm Nebius signs $3 billion deal wit...,Positive,0.003040,0.353695,0.643265
2,Nebius inks $3B AI infrastructure deal with Me...,Neutral,0.161670,0.761175,0.077155
3,"Nebius Reports Bigger Q3 Net Income Loss, Anno...",Negative,0.553435,0.417189,0.029376
4,Why Meta’s Post-Earnings Selloff Is An Early G...,Neutral,0.008171,0.585237,0.406593


7) Sentiment Visualization + Graph-Type Selector (Widgets)

In [116]:
import ipywidgets as widgets

def plot_sentiment(style="Seaborn Bar"):
    counts = sent_df["sentiment"].value_counts().reindex(["Negative","Neutral","Positive"]).fillna(0)
    plt.figure(figsize=(7,5))

    if style == "Seaborn Bar":
        sns.barplot(x=counts.index, y=counts.values, palette="viridis", edgecolor="black")
        plt.title(f"Sentiment Distribution — {company_name}")
        plt.xlabel("Sentiment"); plt.ylabel("Count")

    elif style == "Donut":
        colors = ["#e76f51", "#e9c46a", "#2a9d8f"]
        wedges, texts, autotexts = plt.pie(
            counts.values, labels=counts.index, autopct="%1.1f%%",
            startangle=90, colors=colors, wedgeprops={"linewidth": 2, "edgecolor": "white"}
        )
        centre = plt.Circle((0,0), 0.70, color='white')
        plt.gca().add_artist(centre)
        plt.title(f"Sentiment (Donut) — {company_name}")

    elif style == "Radial":
        vals = counts.values
        labels_r = counts.index
        angles = np.linspace(0, 2*np.pi, len(vals), endpoint=False).tolist()
        vals = np.concatenate((vals, [vals[0]]))
        angles = np.concatenate((angles, [angles[0]]))
        ax = plt.subplot(111, polar=True)
        ax.plot(angles, vals, linewidth=2, color="#2a9d8f")
        ax.fill(angles, vals, alpha=0.3, color="#2a9d8f")
        ax.set_xticks(angles[:-1]); ax.set_xticklabels(labels_r)
        plt.title(f"Sentiment (Radial) — {company_name}")

    elif style == "Glass + Neon":
        from matplotlib.patches import FancyBboxPatch
        ax = plt.gca()
        glass = FancyBboxPatch((0.05,0.05), 0.9, 0.9, boxstyle="round,pad=0.5",
                               fc=(1,1,1,0.08), ec=(0.5,0.5,0.5,0.3), lw=2,
                               transform=ax.transAxes)
        ax.add_patch(glass)
        colors = ["#00F5D4", "#FEE440", "#9B5DE5"]
        sns.barplot(x=counts.index, y=counts.values, palette=colors, edgecolor="white")
        plt.title(f"Sentiment (Glass+Neon) — {company_name}")
        plt.xlabel("Sentiment"); plt.ylabel("Count")

    plt.tight_layout()
    plt.show()

graph_selector = widgets.Dropdown(
    options=["Seaborn Bar", "Donut", "Radial", "Glass + Neon"],
    value="Seaborn Bar",
    description="Chart:",
    style={"description_width": "80px"},
    layout=widgets.Layout(width='350px')
)

ui_box = widgets.VBox([graph_selector])
out = widgets.interactive_output(plot_sentiment, {"style": graph_selector})

display(ui_box, out)


Output()

8) Yahoo Finance History + Forecasting (Prophet)

In [118]:
# ✅ Step 1: Ticker map (expand this as needed)
ticker_map = {
    "Infosys": ["INFY.NS", "INFY"],
    "Tcs": ["TCS.NS", "TCS.BO"],
    "Tata Consultancy Services": ["TCS.NS"],
    "Netflix": ["NFLX"],
    "Reliance Industries": ["RELIANCE.NS"],
    "Reliance": ["RELIANCE.NS"],
}

# ✅ Step 2: Try using ticker map
symbols = ticker_map.get(company_name, [company_name])  # fallback if not in dict

df = None
used_symbol = None

for symbol in symbols:
    try:
        print(f"Trying: {symbol}")
        data = yf.download(symbol, period="3y", interval="1d", auto_adjust=True)
        if not data.empty:
            df = data
            used_symbol = symbol
            break
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

# ✅ Step 3: Final validation
if df is None or df.empty:
    print("❌ No valid stock data found for:", company_name)
else:
    print(f"✅ Using ticker: {used_symbol}")
    df = df.reset_index()[["Date", "Close"]].rename(columns={"Date":"ds", "Close":"y"})
    display(df.tail())


[*********************100%***********************]  1 of 1 completed

Trying: Meta
✅ Using ticker: Meta


Price,ds,y
Ticker,,META
746,2025-11-05,635.950012
747,2025-11-06,618.940002
748,2025-11-07,621.710022
749,2025-11-10,631.760010
750,2025-11-11,627.080017


In [119]:
ticker_map.update({
    "Amazon": ["AMZN"],
    "Google": ["GOOGL"],
    "Alphabet": ["GOOGL"],
    "Meta": ["META"],
    "Facebook": ["META"],
    "Apple": ["AAPL"],
    "Microsoft": ["MSFT"],
    "Tesla": ["TSLA"],
    "Wipro": ["WIPRO.NS"],
    "Hcl": ["HCLTECH.NS"],
})


FORECASTING MODEL 1 — AutoARIMA

In [120]:
series = df["y"]
model_arima = ARIMA(series, order=(5,1,2)).fit()
forecast_arima = model_arima.forecast(30)


WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

  warnings.warn("Maximum Likelihood optimization failed to "



In [121]:
data = df["y"].values.reshape(-1,1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)

X, y = [], []
lookback = 60
for i in range(lookback, len(scaled)):
    X.append(scaled[i-lookback:i])
    y.append(scaled[i])
X, y = np.array(X), np.array(y)

model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(lookback, 1)),
    LSTM(50),
    Dense(1)
])

model_lstm.compile(loss="mse", optimizer="adam")
model_lstm.fit(X, y, epochs=5, batch_size=32, verbose=0)

seq = scaled[-lookback:]
lstm_preds = []

for _ in range(30):
    p = model_lstm.predict(seq.reshape(1, lookback, 1), verbose=0)
    lstm_preds.append(p[0][0])
    seq = np.vstack((seq[1:], p))

forecast_lstm = scaler.inverse_transform(np.array(lstm_preds).reshape(-1,1))


In [122]:
btn_summary = widgets.Button(description="Summary")
btn_news = widgets.Button(description="News")
btn_sent = widgets.Button(description="Sentiment")
btn_arima = widgets.Button(description="ARIMA Forecast")
btn_lstm = widgets.Button(description="LSTM Forecast")

output_area = widgets.Output()

def show_summary(_):
    with output_area:
        clear_output()
        display(HTML("<h2>📘 Company Summary</h2>"))
        if company_info["Logo"]: display(Image(url=company_info["Logo"], width=140))
        display(HTML(f"<p>{company_info['Summary']}</p>"))
        display(pd.DataFrame(company_info, index=[0]))

def show_news(_):
    with output_area:
        clear_output()
        display(HTML("<h2>📰 Latest News</h2>"))
        for n in news_list:
            display(HTML(f'<b>•</b> <a href="{n["link"]}" target="_blank">{n["title"]}</a> <i>({n["source"]})</i><br>'))

def show_sent(_):
    with output_area:
        clear_output()
        display(HTML("<h2>🧠 Sentiment Analysis</h2>"))
        sns.countplot(data=sent_df, x="sentiment")
        plt.show()
        display(sent_df)

def show_arima(_):
    with output_area:
        clear_output()
        display(HTML("<h2>📈 ARIMA Forecast (30 Days)</h2>"))
        plt.figure(figsize=(10,5))
        plt.plot(df.ds, df.y, label="History")
        plt.plot(pd.date_range(df.ds.iloc[-1], periods=30, freq="D"), forecast_arima, label="ARIMA Forecast", color="red")
        plt.legend()
        plt.show()

def show_lstm(_):
    with output_area:
        clear_output()
        display(HTML("<h2>⚡ LSTM Forecast (30 Days)</h2>"))
        plt.figure(figsize=(10,5))
        plt.plot(df.ds, df.y, label="History")
        plt.plot(pd.date_range(df.ds.iloc[-1], periods=30, freq="D"), forecast_lstm, label="LSTM Forecast", color="green")
        plt.legend()
        plt.show()

btn_summary.on_click(show_summary)
btn_news.on_click(show_news)
btn_sent.on_click(show_sent)
btn_arima.on_click(show_arima)
btn_lstm.on_click(show_lstm)

display(widgets.HBox([btn_summary, btn_news, btn_sent, btn_arima, btn_lstm]), output_area)


Output()